In [1]:
%matplotlib inline
import os
import cv2
import json
import torch
import torchvision
from torch import nn
import pandas as pd
import numpy as np
from skimage import color, io
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import MNIST
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, ConcatDataset

In [2]:
##############################################################
# DATA LOADING
##############################################################

#########################
# HUMANS IMAGES
#########################


humans_path = 'pattern_ii_dataset/humans/dataset/synthetic_images'

all_humans = []
    
for root, dirs, files in sorted(os.walk(humans_path)):
    
    for name in files:
        all_humans.append(str(root) + "/" + str(name))
        
print("loaded %d files into human database" % len(all_humans))

#########################
# MONSTERS IMAGES
#########################

monsters_path = 'pattern_ii_dataset/monsters/dataset/synthetic_images'

all_monsters = []
    
for root, dirs, files in sorted(os.walk(monsters_path)):
    
    for name in files:
        all_monsters.append(str(root) + "/" + str(name))
        
print("loaded %d files into monster database" % len(all_monsters))

#########################
# HUMANS ANNOTATIONS
#########################

humans_annotations_path = 'pattern_ii_dataset/humans/dataset/annotations'

all_humans_annotations = []
    
for root, dirs, files in sorted(os.walk(humans_annotations_path)):
    
    for name in files:
        all_humans_annotations.append(str(root) + "/" + str(name))
        
print("loaded %d files into all_humans_annotations database" % len(all_humans_annotations))

print(all_humans_annotations[0])

#########################
# MONSTERS ANNOTATIONS
#########################

monsters_annotations_path = 'pattern_ii_dataset/monsters/dataset/annotations'

all_monsters_annotations = []
    
for root, dirs, files in sorted(os.walk(monsters_annotations_path)):
    
    for name in files:
        all_monsters_annotations.append(str(root) + "/" + str(name))
        
print("loaded %d files into all_monsters_annotations database" % len(all_monsters_annotations))

print(all_monsters_annotations[0])


loaded 36000 files into human database
loaded 6000 files into monster database
loaded 6000 files into all_humans_annotations database
pattern_ii_dataset/humans/dataset/annotations\female/subject_mesh_0001_anno.json
loaded 1000 files into all_monsters_annotations database
pattern_ii_dataset/monsters/dataset/annotations\female/subject_mesh_001_anno.json


In [3]:
##############################################################
# DATA SORTING
##############################################################

##############################
# MONSTERS
##############################

monsters_plain_paths = []
monsters_rgb_paths = []
monsters_texture_paths = []

for monster in all_monsters:
    if "rgb" in monster:
        monsters_rgb_paths.append(monster)
    elif "texture" in monster:
        monsters_texture_paths.append(monster)
    else:
        monsters_plain_paths.append(monster)

##############################
# HUMANS
##############################

humans_plain_paths = []
humans_rgb_paths = []
humans_texture_paths = []

for human in all_humans:
    if "rgb" in human:
        humans_rgb_paths.append(human)
    elif "texture" in human:
        humans_texture_paths.append(human)
    else:
        humans_plain_paths.append(human)
        
##############################
# ANNOTATIONS
##############################


monsters_annotations_df = pd.DataFrame() 

for monster_annotation_path in all_monsters_annotations:
    
    with open (monster_annotation_path, 'r') as file:
    
        # load from json
        monster_annotation = json.load(file)
        sub_df = pd.DataFrame.from_dict(monster_annotation['human_dimensions'], orient='index')
        sub_df = sub_df.transpose()
        
        monsters_annotations_df = monsters_annotations_df.append(sub_df, ignore_index=True)


humans_annotations_df = pd.DataFrame() 

for humans_annotation_path in all_humans_annotations:
    
    with open (humans_annotation_path, 'r') as file:
    
        # load from json
        human_annotation = json.load(file)
        sub_df = pd.DataFrame.from_dict(human_annotation['human_dimensions'], orient='index')
        sub_df = sub_df.transpose()
        
        humans_annotations_df = humans_annotations_df.append(sub_df, ignore_index=True)
    
    
#print(humans_annotations_df.head())
#print(monsters_annotations_df.head())
print(humans_rgb_paths[0])    
print(monsters_rgb_paths[0])    
print(humans_texture_paths[0])    
print(monsters_texture_paths[0])    
print(humans_plain_paths[0])    
print(monsters_plain_paths[0])

pattern_ii_dataset/humans/dataset/synthetic_images\200x200\pose0\female/subject_mesh_w_rgb_0001.png
pattern_ii_dataset/monsters/dataset/synthetic_images\200x200\pose0\female/subject_mesh_w_rgb_001.png
pattern_ii_dataset/humans/dataset/synthetic_images\200x200\pose0\female/subject_mesh_w_texture_0001.png
pattern_ii_dataset/monsters/dataset/synthetic_images\200x200\pose0\female/subject_mesh_w_texture_001.png
pattern_ii_dataset/humans/dataset/synthetic_images\200x200\pose0\female/subject_mesh_0001.png
pattern_ii_dataset/monsters/dataset/synthetic_images\200x200\pose0\female/subject_mesh_001.png


In [4]:
#print(humans_annotations_df.head())
print(monsters_annotations_df.columns)

Index(['chest_circumference', 'height', 'inseam', 'left_arm_length',
       'pelvis_circumference', 'right_arm_length', 'shoulder_width',
       'waist_circumference'],
      dtype='object')


In [5]:
##############################################################
# CONVERT IMAGES TO GRAYSCALE
##############################################################

# reduce number of images for testing purposes
n_images = 1000000

humans_plain = []
humans_texture_grey = []
humans_rgb_grey = []
monsters_plain = []
monsters_texture_grey = []
monsters_rgb_grey = []

##############################
# HUMANS PLAIN
##############################

i = 0

for file_path in humans_plain_paths:
    human_plain = io.imread(file_path)
    
    humans_plain.append(human_plain)
       
    if i == n_images:
        break
        
    i += 1
    

##############################
# HUMANS W TEXTURE BACKGROUND
##############################

i = 0

for file_path in humans_texture_paths:
    human_texture_grey = color.rgb2gray(io.imread(file_path))
    
    humans_texture_grey.append(human_texture_grey)
       
    if i == n_images:
        break
        
    i += 1
    
##############################
# HUMANS W RGB BACKGROUND
##############################

i = 0

for file_path in humans_rgb_paths:
    human_rgb_grey = color.rgb2gray(io.imread(file_path))
    
   
    humans_rgb_grey.append(human_rgb_grey)
       
    if i == n_images:
        break
        
    i += 1
    
##############################
# MONSTERS PLAIN
##############################

i = 0

for file_path in monsters_plain_paths:
    monster_plain = io.imread(file_path)
    
    monsters_plain.append(monster_plain)
       
    if i == n_images:
        break
        
    i += 1

##############################
# MONSTERS W TEXTURE BACKGROUND
##############################

i = 0

for file_path in monsters_texture_paths:
    monster_texture_grey = color.rgb2gray(io.imread(file_path))
      
    monsters_texture_grey.append(monster_texture_grey)
       
    if i == n_images:
        break
        
    i += 1
    

##############################
# MONSTERS W RGB BACKGROUND
##############################

i = 0

for file_path in monsters_rgb_paths:
    monster_rgb_grey = color.rgb2gray(io.imread(file_path))
    
    monsters_rgb_grey.append(monster_rgb_grey)
       
    if i == n_images:
        break
        
    i += 1

    
#######################################################################
# DISPLAY GREY IMAGES -
#######################################################################

#cv2.imshow("humans_plain", humans_plain[0])
#cv2.imshow("humans_rgb_grey", humans_rgb_grey[0])
#cv2.imshow("humans_texture_grey", humans_texture_grey[0])   
#cv2.imshow("monsters_plain", monsters_plain[0])     
#cv2.imshow("monsters_rgb_grey", monsters_rgb_grey[0])
#cv2.imshow("monsters_texture_grey", monsters_texture_grey[5])
#cv2.waitKey(0)

print("finished")

finished


In [6]:
##############################################################
# CREATE DATA FRAME OF IMAGE VECTORS AND ANNOTATIONS
##############################################################

##############################
# HUMAN PLAIN DATA FRAME
##############################


# tensor of images w.o. annotations
humans_plain_image_tensor = torch.tensor(humans_plain)

# tensor of annotations
human_annotation_tensor = torch.tensor(humans_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
human_annotation_tensor = torch.cat((human_annotation_tensor,human_annotation_tensor),0)

# tensor of image and annotations
humans_plain_full_tensor = torch.cat((humans_plain_image_tensor,human_annotation_tensor),1)


##############################
# HUMAN RGB DATA FRAME
##############################


# tensor of images w.o. annotations
humans_rgb_image_tensor = torch.tensor(humans_rgb_grey)

# tensor of annotations
human_annotation_tensor = torch.tensor(humans_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
human_annotation_tensor = torch.cat((human_annotation_tensor,human_annotation_tensor),0)

# tensor of image and annotations
humans_rgb_full_tensor = torch.cat((humans_rgb_image_tensor,human_annotation_tensor),1)


##############################
# HUMAN TEXTURE DATA FRAME
##############################


# tensor of images w.o. annotations
humans_texture_image_tensor = torch.tensor(humans_texture_grey)

# tensor of annotations
human_annotation_tensor = torch.tensor(humans_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
human_annotation_tensor = torch.cat((human_annotation_tensor,human_annotation_tensor),0)

# tensor of image and annotations
humans_texture_full_tensor = torch.cat((humans_texture_image_tensor,human_annotation_tensor),1)


##############################
# MONSTERS PLAIN DATA FRAME
##############################


# tensor of images w.o. annotations
monsters_plain_image_tensor = torch.tensor(monsters_plain)

# tensor of annotations
monster_annotation_tensor = torch.tensor(monsters_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
monster_annotation_tensor = torch.cat((monster_annotation_tensor,monster_annotation_tensor),0)

# tensor of image and annotations
monsters_plain_full_tensor = torch.cat((monsters_plain_image_tensor,monster_annotation_tensor),1)


##############################
# MONSTERS RGB DATA FRAME
##############################


# tensor of images w.o. annotations
monster_rgb_image_tensor = torch.tensor(monsters_rgb_grey)

# tensor of annotations
monster_annotation_tensor = torch.tensor(monsters_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
monster_annotation_tensor = torch.cat((monster_annotation_tensor,monster_annotation_tensor),0)

# tensor of image and annotations
monsters_rgb_full_tensor = torch.cat((monster_rgb_image_tensor,monster_annotation_tensor),1)


##############################
# MONSTERS TEXTURE DATA FRAME
##############################


# tensor of images w.o. annotations
monster_texture_image_tensor = torch.tensor(monsters_texture_grey)

# tensor of annotations
monster_annotation_tensor = torch.tensor(monsters_annotations_df.values)

# copy annotations - pose0 and pose1 have the same annotations
monster_annotation_tensor = torch.cat((monster_annotation_tensor,monster_annotation_tensor),0)

# tensor of image and annotations
monsters_texture_full_tensor = torch.cat((monster_texture_image_tensor,monster_annotation_tensor),1)

##############################
# HUMAN AND MONSTERS PLAIN
##############################

humans_monsters_plain_full_tensor = torch.cat((humans_plain_full_tensor, monsters_plain_full_tensor),0)

##############################
# HUMAN AND MONSTERS RGB
##############################

humans_monsters_rgb_full_tensor = torch.cat((humans_rgb_full_tensor, monsters_rgb_full_tensor),0)

##############################
# HUMAN AND MONSTERS TEXTURE
##############################

humans_monsters_texture_full_tensor = torch.cat((humans_texture_full_tensor, monsters_texture_full_tensor),0)

##############################
# SAVE TENSORS TO FILES
##############################

torch.save(humans_plain_full_tensor, 'tensors/humans_plain_full_tensor.pt')
print("humans_plain_full_tensor.pt saved")
torch.save(humans_rgb_full_tensor, 'tensors/humans_rgb_full_tensor.pt')
print("humans_rgb_full_tensor.pt saved")
torch.save(humans_texture_full_tensor, 'tensors/humans_texture_full_tensor.pt')
print("humans_texture_full_tensor.pt saved")

torch.save(monsters_plain_full_tensor, 'tensors/monsters_plain_full_tensor.pt')
print("monsters_plain_full_tensor.pt saved")
torch.save(monsters_rgb_full_tensor, 'tensors/monsters_rgb_full_tensor.pt')
print("monsters_rgb_full_tensor.pt saved")
torch.save(monsters_texture_full_tensor, 'tensors/monsters_texture_full_tensor.pt')
print("monsters_texture_full_tensor.pt saved")

torch.save(humans_monsters_plain_full_tensor, 'tensors/humans_monsters_plain_full_tensor.pt')
print("humans_monsters_plain_full_tensor.pt saved")
torch.save(humans_monsters_rgb_full_tensor, 'tensors/humans_monsters_rgb_full_tensor.pt')
print("humans_monsters_rgb_full_tensor.pt saved")
torch.save(humans_monsters_texture_full_tensor, 'tensors/humans_monsters_texture_full_tensor.pt')
print("humans_monsters_texture_full_tensor.pt saved")

RuntimeError: torch.cat(): Tensors must have same number of dimensions: got 3 and 2

In [ ]:
#Index(['chest_circumference', 'height', 'inseam', 'left_arm_length',
#       'pelvis_circumference', 'right_arm_length', 'shoulder_width',
#       'waist_circumference'],
#      dtype='object')